# This is a Python function that computes the internal-tide wave drag coefficient at each grid point

Input: 
- Location: lon, lat
- wave frequency (tidal constituent)

Steps:
1. Get the Coriolis frequency at this lat
2. Figure out the index of lon and lat in the topog dataset
3. Get the local depth at this loc
4. Get the bottom stratification at this loc
5. Estimate the horizontal wavelength of mode-1 $M_2$ internal tides
6. Figure out the index range to sample the topog according to the hori. wavelength
7. Sample the topography (plotting optional)
8. Construct a 2D filter
9. Apply the filter and obtain the filtered topography (plotting optional)
10. Perform the spectral analysis
11. Compare the 1D spectral slope with that from G2010
12. Compute the drag coefficient

Output: 
- Drag coefficients: $\sigma_x$, $\sigma_y$

In [1]:
%matplotlib inline

import xrft
import math
import scipy.io
import numpy as np
import xarray as xr
import netCDF4 as nc
import cmocean as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from gsw import f
from matplotlib.colors import LogNorm
from scipy.fftpack import fft2, fftn, fftshift

import warnings
warnings.filterwarnings('ignore')

In [2]:
def drag_coeff(lon,lat,omega):

    
    # estimate the mode-1 wavelength
    f_loc = f(lat)
    N_loc = get_N_clim(lon,lat) * 2 * np.pi
    topog = xr.open_dataset('/g/data/nm03/lxy581/synbath/SYNBATH.nc')
    depth = topog.z
    H_loc = -depth.interp(lat=lat,lon=lon).values
    kh = estimate_kh(omega,f_loc,N_loc,H_loc)
    print('The horizontal wavelength of the mode-1 M2 internal tides is approx. %d km. \n' % (2 * np.pi / kh / 1000))

    
    # compute the sampling step in deg based on the wavelength of mode-1 waves
    delta_lon, delta_lat = get_delta(lat)
    print('The distance per degree lon is %.1f km.' % delta_lon)
    print('The distance per degree lat is %.1f km. \n' % delta_lat)
    step_lon, step_lat = get_radius_in_deg(kh,lon,lat,delta_lon,delta_lat)
    step = max(step_lon, step_lat)
    print('The sampling step is %.1f degree.' % step)
    

    
    # sample the topography
    topog_sample = depth.where((topog.lon>lon-step) & (topog.lon<lon+step) & (topog.lat>lat-step) & (topog.lat<lat+step), drop=True)
    print('Shape of the sampled topog: \n', topog_sample.shape)

    # topog_sample -= topog_sample.mean(skipna=True)
    # print("Plotting sampled topography")
    # fig = plt.figure(figsize = (8, 6))
    # levels = np.arange(-3000,3000+1000,1000)
    # xr.plot.pcolormesh(topog_sample, vmin=-3000, vmax=3000, levels=61, extend='both', cmap='bwr', cbar_kwargs={'label': "Depth (m)", 'ticks': levels})
    # plt.title('Ocean depth, demeaned (m)');

    
    # construct a 2D filter and filter the topog
    #topog_filt = filtering(topog_sample)
    #print('Shape of the filtered topog: ', topog_filt.shape)

    
    # perform spectral analysis
    print('Spectral analysis... \n')
    #topog_spd_1d, kh_1d, topog_spd_2d, kx_2d, ky_2d, dk, dl, dy, dx = spectral_analysis(topog_sample,lat,delta_lon,delta_lat)
    topog_spd_2d, dx, dy = fft_topog(topog_sample,delta_lon,delta_lat,k_grid_units=False)
    print('Azimuthal summing... \n')
    topog_spd_1d = azimuthal_sum(topog_spd_2d)

    
    # check Parseval's theorem
    print('Checking Parsevals theorem...')
    dkx, dky = np.max(np.diff(topog_spd_2d.kx)), np.max(np.diff(topog_spd_2d.ky))
    dkh = np.max(np.diff(topog_spd_1d.k))
    fft2dsum = topog_spd_2d.sum().sum()*dkx*dky
    fft1dsum = topog_spd_1d.sum()*dkh
    print('2D sum (k-space)     = %3.2e' % fft2dsum)
    print('Radial sum (k-space) = %3.2e' % fft1dsum)
    var_int_x = np.nansum(topog_sample **2 * dy * dx)
    print('2D sum (x-space)     = %3.2e \n' % var_int_x)
    
    # var_int_k = np.nansum(topog_spd_2d * dk * dl)
    # print('2D spectrum integrated in wavenumber space: ', var_int_k)
    
    
    # # compare the slope
    # mat_ga2010 = scipy.io.loadmat('/home/581/lxy581/raijin_home_2019-11-15/lxy581/lw/goff.mat')
    # nu_ga2010  = mat_ga2010['nu']
    # lon_ga2010 = mat_ga2010['lon'][0,:] # 1440
    # lat_ga2010 = mat_ga2010['lat'][:,0] # 580
    # xid_ga, yid_ga = 600, 480
    # lon_ga2010[xid_ga], lat_ga2010[yid_ga]

    # mat_g2010 = scipy.io.loadmat('/home/581/lxy581/raijin_home_2019-11-15/lxy581/lw/goff2010_grids/goff2010_nu.mat')
    # nu_g2010  = mat_g2010['nu'] #2250 x 5400
    # lon_g2010 = mat_g2010['x'][0,:] # 5400
    # lat_g2010 = mat_g2010['y'][0,:] # 2250
    # xid_g, yid_g = 2250, 1875
    # lon_g2010[xid_g], lat_g2010[yid_g]

    # slope_ga = -2*(nu_ga2010[yid_ga,xid_ga]+1)+1
    # slope_g  = -2*(nu_g2010[yid_g,xid_g]+1)+1
    # kh_abyssal = kh_1d[30:1000]

    # sp_abyssal_ga = 5e-5*kh_abyssal ** slope_ga
    # sp_abyssal_g  = 3e-4*kh_abyssal ** slope_g
    # fig = plt.figure(figsize = (8,6))
    # plt.loglog(kh_abyssal, sp_abyssal_g, 'r--', label='G2010')
    # plt.loglog(kh_abyssal, sp_abyssal_ga, 'b--', label='GA2010')
    # plt.loglog(kh_1d,topog_spd_1d,'k-', label='SYNBATH spectrum')
    # plt.legend(loc=0)
    # plt.xlabel('Horizontal wavenumber [cpm]')
    # plt.ylabel('[m$^{2}$ cpm$^{-1}$]')
    # plt.grid(which='both', axis='both')


    
    # compute the drag coefficient
    print('Computing the drag coeff...')
    A_loc = 2 * step * delta_lon * 1e+3 * 2 * step * delta_lat * 1e+3 
    sigma_x, sigma_y = compute_drag_coeff(A_loc,N_loc,H_loc,topog_spd_2d)
    print('Drag coefficient (x-dir): %3.2e' % sigma_x)
    print('Drag coefficient (y-dir): %3.2e' % sigma_y)

    
    return sigma_x, sigma_y

In [3]:
def get_N_clim(lon,lat):
    
    Nbot_data = xr.open_dataset('/g/data/nm03/lxy581/WOA18/Nbot_1000m_woa18.nc')

    # Read bottom N and convert the unit to rad/s
    Nbot_1km = Nbot_data.Nbot * 2 * np.pi

    # Use the 2D interpolation to find the bottom N
    N_clim = Nbot_1km.interp(lat=lat,lon=lon).values
    
    return N_clim

In [4]:
def estimate_kh(omega,f_loc,N_loc,H_loc):
    
    kh = np.sqrt((omega**2-f_loc**2)*np.pi**2/N_loc**2/H_loc**2)
    
    return kh

In [5]:
def get_delta(lat):

    # compute the distance in km at this lon and lat
    delta_lon = 2 * np.pi * (6371 * np.cos(lat*np.pi/180)) / 360
    delta_lat = 1 * np.pi * 6371 / 180
    
    return delta_lon, delta_lat

In [6]:
def get_radius_in_deg(kh,lon,lat,delta_lon,delta_lat):
    
    radius = 2 * np.pi / kh / 1000 / 2

    step_lon  = round(radius / delta_lon, 1)
    step_lat  = round(radius / delta_lat, 1)
    
    return step_lon, step_lat

In [7]:
def filtering(topog_sample):
    
    ydim, xdim = topog_sample.dims
    nx = topog_sample[xdim].size
    ny = topog_sample[ydim].size
    win_2d = np.full((ny,nx), np.nan)
    radi = topog_sample.shape[0] / 2
    for i in range(topog_sample.shape[0]):
      for j in range(topog_sample.shape[0]):
        win_2d[j,i] = 1 - ( ((i-radi)/radi)**2 + ((j-radi)/radi)**2 )
    win_2d[win_2d<0]=0

    fac = np.sqrt(win_2d.size / np.sum(win_2d**2))
    win_2d = win_2d * fac
    win_2d_xr = xr.DataArray(win_2d, coords={'y': np.arange(ny), 'x': np.arange(nx)}, dims=["y", "x"])
    
    topog_filt = np.array(topog_sample) * np.array(win_2d_xr)
    topog_filt = xr.DataArray(topog_filt, coords=topog_sample.coords, dims=topog_sample.dims)

    # print("Plotting filtered topography...")
    # fig = plt.figure(figsize = (8, 6))
    # levels = np.arange(-3000,3000+1000,1000)
    # xr.plot.pcolormesh(topog_filt, vmin=-3000, vmax=3000, levels=61, extend='both', cmap='bwr', cbar_kwargs={'label': "Depth (m)", 'ticks': levels})
    # plt.title('Ocean depth, demeaned & filtered (m)');
    
    return topog_filt

In [8]:
def spectral_analysis(topog,lat,delta_lon,delta_lat):

    ydim, xdim = topog.dims
    nx = topog[xdim].size
    ny = topog[ydim].size
    dx = np.mean(np.diff(topog[xdim])) * delta_lon * 1e+3
    dy = np.mean(np.diff(topog[ydim])) * delta_lat * 1e+3
    print('dx, dy = ', dx, dy)

    topog_fft = np.fft.fft2(topog)
    topog_fft = topog_fft * dx * dy / (8.0*(np.pi)**2*nx*ny)
    topog_spd = (topog_fft*topog_fft.conjugate()).real

    kx = np.zeros(nx) 
    for i in range(nx):
        kx[i] = i
        if kx[i] > nx / 2:
            kx[i] = nx - i
    nx_fold = int(nx/2 + 1)
    kx_2d = kx[0:nx_fold]/nx/dx

    ky = np.zeros(ny) 
    for j in range(ny):
        ky[j] = j
        if ky[j] > ny / 2:
            ky[j] = ny - j
    ny_fold = int(ny/2 + 1)
    ky_2d = ky[0:ny_fold]/ny/dy

    dk = kx_2d[1] - kx_2d[0]
    dl = ky_2d[1] - ky_2d[0]
    
    kh = np.zeros((ny,nx))
    for j in range(ny):
        for i in range(nx):
            kh[j,i] = np.sqrt(kx[i]**2 + ky[j]**2)
    kh_max = round(np.sqrt( (nx/2)**2 + (ny/2)**2 ))
       
    topog_spd_1d   = np.zeros((kh_max + 1))
    topog_spd_fold = np.zeros((round(np.max(ky))+1,round(np.max(kx))+1))
    for j in range(ny):
        for i in range(nx):
            jj = round(ky[j])
            ii = round(kx[i])
            ik = round(kh[j,i])
            topog_spd_1d[ik]      = topog_spd_1d[ik] + topog_spd[j,i] 
            topog_spd_fold[jj,ii] = topog_spd_fold[jj,ii]+topog_spd[j,i]
            # if jj==1 and ii==1:
            #     print('j,i,jj,ii,topog_spd_fold: ',j,i,jj,ii,topog_spd[j,i],topog_spd_fold[jj,ii])
    
    dh = np.sqrt( dx**2 + dy**2 ) 
    topog_spd_1d = topog_spd_1d * nx * dh
    kh_1d = np.arange(kh_max+1) / nx / dh

    topog_spd_2d = topog_spd_fold * nx * dx * ny * dy 
    print('Check shape: ', topog_spd_2d.shape, kx_2d.shape, ky_2d.shape)
    
    return topog_spd_1d, kh_1d, topog_spd_2d, kx_2d, ky_2d, dk, dl, dy, dx

In [9]:
def fft_topog(topog,delta_lon,delta_lat,k_grid_units=True):

    ydim, xdim = topog.dims
    nx = topog[xdim].size
    ny = topog[ydim].size
    dx = np.mean(np.diff(topog[xdim]))*delta_lon*1e+3
    dy = np.mean(np.diff(topog[ydim]))*delta_lat*1e+3

    # demean
    topog -= topog.mean(skipna=True)
    
    # windowing
    topog_filt = filtering(topog)

    # FFT
    topog_fft = fft2(topog.values)
    topog_spd = (topog_fft*topog_fft.conjugate()).real
    topog_spd[0,0] = np.nan          # nan at removed zero frequency
    topog_spd = fftshift(topog_spd)  # put zero wavenumber in array centre
    topog_spd *= dx*dy

    if (k_grid_units):
        # k in units cycles/dx:
        topog_spd = xr.DataArray(topog_spd, dims=['ky','kx'], 
                                 coords={'ky': np.linspace(-0.5, 0.5+(ny%2-1)/ny, num=ny), 
                                         'kx': np.linspace(-0.5, 0.5+(nx%2-1)/nx, num=nx)},
                                 attrs={'long_name': 'wavenumber spectrum in grid units'})
        topog_spd.kx.attrs['units'] = 'cycles/dx'
        topog_spd.ky.attrs['units'] = 'cycles/dy'
        topog_spd.kx.attrs['long_name'] = 'x wavenumber'
        topog_spd.ky.attrs['long_name'] = 'y wavenumber'

        # No rescaling

    else:
        # k in units cycles/(units of dx)
        topog_spd = xr.DataArray(topog_spd, dims=['ky','kx'], 
                                 coords={'ky': np.linspace(-0.5, 0.5+(ny%2-1)/ny, num=ny)*(2*np.pi/dy),
                                         'kx': np.linspace(-0.5, 0.5+(nx%2-1)/nx, num=nx)*(2*np.pi/dx)},
                                 attrs={'long_name': 'wavenumber spectrum in grid units'})
        topog_spd.kx.attrs['units'] = 'radians/meters'
        topog_spd.ky.attrs['units'] = 'radians/meters'
        topog_spd.kx.attrs['long_name'] = 'x wavenumber'
        topog_spd.ky.attrs['long_name'] = 'y wavenumber'

        # Rescale to satisfy Parseval's theorem:
        topog_spd /= 4*np.pi**2./dx/dy

    return topog_spd, dx, dy

In [10]:
def azimuthal_sum(topog_spd):  
    
    # following DurranWeynMenchaca2017a http://dx.doi.org/10.1175/MWR-D-17-0056.1
    # omits zero wavenumber
    dkh = np.max([np.max(np.diff(topog_spd.kx)), np.max(np.diff(topog_spd.ky))])
    dkmin = np.min([np.min(np.diff(topog_spd.kx)), np.min(np.diff(topog_spd.ky))])
    
    # extends sqrt(2) times further to get into corners
    Nmax = int(np.ceil(np.sqrt(2)*max(topog_spd.shape)/2))  
    kp = dkh*range(1,Nmax+1)
    
    # number of wavenumber points in each annulus (C in DurranWeynMenchaca2017a)
    C = 0.0*kp  
    fftradial = 0.0*kp
    radius = np.sqrt(topog_spd.kx**2+topog_spd.ky**2)
    ones = 1 + 0*topog_spd
    
    # sum in each annulus
    for i,k in enumerate(kp):
        fftradial[i] = topog_spd.where(radius>=k-dkh/2).where(radius<k+dkh/2).sum()
        C[i] = ones.where(radius>k-dkh/2).where(radius<=k+dkh/2).sum()

    # scale as in eq (24) (assuming scaling in eq (22) is already done)
    fftradial *= dkmin  

    # eq (26): compensate for number of (k,l) pairs in each annulus
    # Parseval's theorem no longer exactly holds (p 3905)
    C = np.where(C==0, 1, C)  # ensures no division by zero (fftradial=0 there anyway)
    fftradial *= 2.0*np.pi*kp/C/dkmin

    fftradial = xr.DataArray(fftradial, dims=['k'], coords={'k': kp})
    fftradial.k.attrs['units'] = topog_spd.kx.attrs['units']
    fftradial.k.attrs['long_name'] = 'wavenumber magnitude'

    # Truncate spectrum at Nyquist frequency (high k's in corners are anisotropically sampled):
    # Also breaks Parseval's theorem
    kminmax = np.min([np.max(topog_spd.kx), np.max(topog_spd.ky)])
    fftradial = fftradial.sel(k=slice(0.,kminmax))    

    return fftradial

In [11]:
def compute_drag_coeff(A,N,H,topog_spd_2d):
    
    const = N / (4 * np.pi**2 * H * A)
    
    kx_2D, ky_2D = np.meshgrid(topog_spd_2d.kx,topog_spd_2d.ky,indexing='xy')
    dkx, dky = np.max(np.diff(topog_spd_2d.kx)), np.max(np.diff(topog_spd_2d.ky))
    int_x = topog_spd_2d * kx_2D**2 / np.sqrt(kx_2D**2 + ky_2D**2) * dkx * dky
    int_y = topog_spd_2d * ky_2D**2 / np.sqrt(kx_2D**2 + ky_2D**2) * dkx * dky
    sigma_x = const * np.nansum(int_x[:])
    sigma_y = const * np.nansum(int_y[:])
    
    return sigma_x, sigma_y

### Testing cell below

Provide input details for the example: <br>
* lon 30$^{\circ}$W <br>
* lat 50$^{\circ}$N
* tidal frequency, in this case, $\omega_{M_2}$ 

In [12]:
lon = -30
lat = 50
omega = 2 * np.pi / (12*3600)

Call the function 

In [13]:
sigma_x, sigma_y = drag_coeff(lon,lat,omega)

The horizontal wavelength of the mode-1 M2 internal tides is approx. 374 km. 

The distance per degree lon is 71.5 km.
The distance per degree lat is 111.2 km. 

The sampling step is 2.6 degree.
Shape of the sampled topog: 
 (1248, 1248)
Spectral analysis... 

Azimuthal summing... 

Checking Parsevals theorem...
2D sum (k-space)     = 7.20e+16
Radial sum (k-space) = 7.13e+16
2D sum (x-space)     = 7.20e+16 

Computing the drag coeff...
Drag coefficient (x-dir): 1.36e-06
Drag coefficient (y-dir): 1.02e-06
